In [1]:
import tvm
from tvm import relay
import tvm.testing
from tvm.contrib import graph_executor
from tvm.relay import ExprMutator
import numpy as np

import torch

CLASSES = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
INPUT_PREFIX = "input"
BATCH_SIZE = 32
DIFF_THRESHOLD = 1e-5

def execute_tvm_model(relay_model):
#    relay_model.set_input(f"{INPUT_PREFIX}_0", images)
    relay_model.run()
    return torch.from_numpy(
        relay_model.get_output(0).asnumpy()
    )

def compile_into_tvm(mod, params):
    with tvm.transform.PassContext(opt_level=0):
        relay_graph, relay_lib, relay_params = relay.build(
            mod, target="llvm", params=params
        )
    relay_model = graph_executor.create(relay_graph, relay_lib, tvm.cpu(0))
    relay_model.set_input(**relay_params)
    return relay_model

In [5]:
with open("/home/julien/3la-evaluation/flexmatch/tests/models/conv2d-im2col.relay", "r") as fp:
    new_mod_text = fp.read()

new_mod = tvm.parser.fromtext(new_mod_text)

dev = tvm.cpu(0)
params = {"data": tvm.nd.array(np.random.uniform(size=(1,3,32,32)).astype("float32"), dev), 
           "weights": tvm.nd.array(np.random.uniform(size=(2,3,16,16)).astype("float32"), dev)}

mod2 =compile_into_tvm(new_mod, params)
print(execute_tvm_model(mod2))

AttributeError: AttributeError: relay.attrs.StridedSliceAttrs: Cannot find required field 'axes' during initialization. If the key is defined check that its type matches the declared type.